# Azure OpenAI Python Patterns: Step-by-Step Examples

**Original Code by Dave Ebbelaar** - Extended for Azure OpenAI by Arturo Quiroga

This notebook provides a step-by-step walkthrough of four foundational patterns for using Azure OpenAI in Python. Each section is broken down into multiple cells to clarify the purpose and logic of each file and code segment.

**Patterns covered:**
- Basic Chat Completion (from `1-basic.py`)
- Structured Output (from `2-structured.py`)
- Function Calling / Tools (from `3-tools.py`)
- Retrieval-Augmented Generation (from `4-retrieval.py`)

---

## 1. Basic Chat Completion (1-basic.py)
This section demonstrates a simple chat completion using Azure OpenAI.

In [1]:
# Import required utilities and validate environment
import sys, os
# In a notebook, __file__ is not defined; use os.getcwd() instead
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from azure_utils import get_azure_openai_client, get_deployment_name, validate_environment
if not validate_environment():
    raise RuntimeError("Environment validation failed. Please check your .env configuration.")


In [2]:
# Initialize Azure OpenAI client
client = get_azure_openai_client()
deployment_name = get_deployment_name("gpt4")

2025-07-29 12:23:31,982 - INFO - Azure OpenAI client initialized with API key authentication


In [3]:
# Make a simple chat completion request
print("Azure OpenAI Response:")
print("=" * 50)
try:
    completion = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "You're a helpful assistant."},
            {"role": "user", "content": "Write a limerick about the Python programming language."},
        ],
        temperature=0.7,
        max_tokens=1000
    )
    response = completion.choices[0].message.content
    print(response)
except Exception as e:
    print(f"Error: {e}")
    print("Make sure your Azure OpenAI configuration is correct in .env file")


Azure OpenAI Response:


2025-07-29 12:23:33,600 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


There once was a coder in Brighton  
Whose code was as smooth as pure Python  
With syntax so neat,  
Indentation a treat,  
Debugging became quite a light one!


---

## 2. Structured Output (2-structured.py)
This section demonstrates how to use Pydantic models to get structured responses from Azure OpenAI.

In [4]:
# Define a Pydantic model for structured output
from pydantic import BaseModel
from typing import List
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: List[str]

In [5]:
# Call the model with structured output
print("Extracted Calendar Event:")
print("=" * 30)
try:
    completion = client.beta.chat.completions.parse(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "Extract the event information."},
            {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
        ],
        response_format=CalendarEvent,
        temperature=0.0,
    )
    event = completion.choices[0].message.parsed
    print(f"Event Name: {event.name}")
    print(f"Date: {event.date}")
    print(f"Participants: {', '.join(event.participants)}")
    print(f"\nIndividual field access:")
    print(f"- Name: {event.name}")
    print(f"- Date: {event.date}")
    print(f"- Participants: {event.participants}")
except Exception as e:
    print(f"Error: {e}")
    print("Make sure your Azure OpenAI configuration is correct in .env file")


Extracted Calendar Event:


2025-07-29 12:23:34,190 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Event Name: Science Fair
Date: Friday
Participants: Alice, Bob

Individual field access:
- Name: Science Fair
- Date: Friday
- Participants: ['Alice', 'Bob']


---

## 3. Function Calling / Tools (3-tools.py)
This section demonstrates how to enable the model to call Python functions (tools) and integrate external APIs.

In [6]:
# Define a tool function for weather
import requests
def get_weather(latitude: float, longitude: float) -> dict:
    try:
        response = requests.get(
            f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m'
        )
        response.raise_for_status()
        data = response.json()
        return data.get('current', {})
    except Exception as e:
        return {'error': f'Failed to fetch weather data: {str(e)}'}

In [7]:
# Tool use with Azure OpenAI: Function Calling Example
import json
import pprint
print("Tool Use with Azure OpenAI (Function Calling)")
print("=" * 50)
try:
    tools = [
        {
            'type': 'function',
            'function': {
                'name': 'get_weather',
                'description': 'Get current temperature for provided coordinates in celsius.',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'latitude': {'type': 'number'},
                        'longitude': {'type': 'number'},
                    },
                    'required': ['latitude', 'longitude'],
                    'additionalProperties': False,
                },
                'strict': True,
            },
        }
    ]
    system_prompt = 'You are a helpful weather assistant.'
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': "What's the weather like in Paris today?"},
    ]
    print("Calling Azure OpenAI with tool definition...")
    completion = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        temperature=0.1,
    )
    print("Model response received.")
    assistant_message = completion.choices[0].message
    if assistant_message.tool_calls:
        print(f"Model wants to call {len(assistant_message.tool_calls)} tool(s)")
        messages.append(assistant_message)
        for tool_call in assistant_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f"Executing function: {function_name}")
            print(f"Arguments: {function_args}")
            # Call the function as in the script
            result = get_weather(**function_args)
            print(f"Function result: {result}")
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })
        print("\nGetting final response with function results...")
        # Optionally, you can call a follow-up completion here for a final answer
    else:
        print("No tool calls were made by the model.")
        print(f"Direct response: {assistant_message.content}")
except Exception as e:
    print(f"Error: {e}")
    print("Make sure your Azure OpenAI configuration is correct in .env file")


Tool Use with Azure OpenAI (Function Calling)
Calling Azure OpenAI with tool definition...


2025-07-29 12:23:34,783 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Model response received.
Model wants to call 1 tool(s)
Executing function: get_weather
Arguments: {'latitude': 48.8566, 'longitude': 2.3522}
Function result: {'time': '2025-07-29T16:15', 'interval': 900, 'temperature_2m': 20.0, 'wind_speed_10m': 7.4}

Getting final response with function results...


---

## 4. Retrieval-Augmented Generation (4-retrieval.py)
This section demonstrates how to connect the model to a knowledge base for question answering.

In [8]:
# Define a simple knowledge base search tool
import json
import os

def search_kb(question: str) -> dict:
    try:
        # In a notebook, __file__ is not defined; use os.getcwd() instead
        current_dir = os.getcwd()
        kb_path = os.path.join(current_dir, 'kb.json')
        with open(kb_path, 'r') as f:
            kb_data = json.load(f)
        return kb_data
    except Exception as e:
        return {'error': f'Failed to load knowledge base: {str(e)}'}


In [9]:
# Retrieval with Azure OpenAI: Knowledge Base Tool Example
import json
import pprint
print("Azure OpenAI Knowledge Base Assistant")
print("=" * 50)
try:
    tools = [
        {
            'type': 'function',
            'function': {
                'name': 'search_kb',
                'description': "Get the answer to the user's question from the knowledge base about our e-commerce store.",
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'question': {'type': 'string'},
                    },
                    'required': ['question'],
                    'additionalProperties': False,
                },
                'strict': True,
            },
        }
    ]
    system_prompt = 'You are a helpful assistant that answers questions from the knowledge base about our e-commerce store.'
    kb_questions = [
        "What is the return policy?",
        "Do you ship internationally?",
        "What payment methods do you accept?",
        "How do I track my order?"
    ]
    for question in kb_questions:
        print(f"\nAsking: '{question}'")
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': question},
        ]
        completion = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            tools=tools,
            temperature=0.1
        )
        assistant_message = completion.choices[0].message
        if assistant_message.tool_calls:
            print("Model is searching knowledge base...")
            messages.append(assistant_message)
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                print(f"Executing function: {function_name}")
                print(f"Arguments: {function_args}")
                result = search_kb(**function_args)
                print(f"Function result: {result}")
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result)
                })
            # Optionally, you can call a follow-up completion here for a final answer
        else:
            print("Model responding directly (no KB search needed)")
            print(f"Direct response: {assistant_message.content}")
except Exception as e:
    print(f"Error: {e}")
    print("Make sure your Azure OpenAI configuration is correct in .env file")


Azure OpenAI Knowledge Base Assistant

Asking: 'What is the return policy?'


2025-07-29 12:23:36,050 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Model is searching knowledge base...
Executing function: search_kb
Arguments: {'question': 'What is the return policy?'}
Function result: {'records': [{'id': 1, 'question': 'What is the return policy?', 'answer': 'Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.'}, {'id': 2, 'question': 'Do you ship internationally?', 'answer': 'Yes, we ship to over 50 countries worldwide. International shipping typically takes 7-14 business days and costs vary by destination. Please note that customs fees may apply.'}, {'id': 3, 'question': 'What payment methods do you accept?', 'answer': 'We accept Visa, Mastercard, American Express, PayPal, and Apple Pay. All payments are processed securely through our encrypted payment system.'}, {'id': 4, 'question': 'How do I track my order?', 'answer': "Once your order ships, you'll receive a tracking number via email. You can also log into your account to v

2025-07-29 12:23:36,531 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Model is searching knowledge base...
Executing function: search_kb
Arguments: {'question': 'Do you ship internationally?'}
Function result: {'records': [{'id': 1, 'question': 'What is the return policy?', 'answer': 'Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.'}, {'id': 2, 'question': 'Do you ship internationally?', 'answer': 'Yes, we ship to over 50 countries worldwide. International shipping typically takes 7-14 business days and costs vary by destination. Please note that customs fees may apply.'}, {'id': 3, 'question': 'What payment methods do you accept?', 'answer': 'We accept Visa, Mastercard, American Express, PayPal, and Apple Pay. All payments are processed securely through our encrypted payment system.'}, {'id': 4, 'question': 'How do I track my order?', 'answer': "Once your order ships, you'll receive a tracking number via email. You can also log into your account to

2025-07-29 12:23:37,353 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Model is searching knowledge base...
Executing function: search_kb
Arguments: {'question': 'What payment methods do you accept?'}
Function result: {'records': [{'id': 1, 'question': 'What is the return policy?', 'answer': 'Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.'}, {'id': 2, 'question': 'Do you ship internationally?', 'answer': 'Yes, we ship to over 50 countries worldwide. International shipping typically takes 7-14 business days and costs vary by destination. Please note that customs fees may apply.'}, {'id': 3, 'question': 'What payment methods do you accept?', 'answer': 'We accept Visa, Mastercard, American Express, PayPal, and Apple Pay. All payments are processed securely through our encrypted payment system.'}, {'id': 4, 'question': 'How do I track my order?', 'answer': "Once your order ships, you'll receive a tracking number via email. You can also log into your acc

2025-07-29 12:23:37,882 - INFO - HTTP Request: POST https://aq-ai-foundry-sweden-central.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Model is searching knowledge base...
Executing function: search_kb
Arguments: {'question': 'How do I track my order?'}
Function result: {'records': [{'id': 1, 'question': 'What is the return policy?', 'answer': 'Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.'}, {'id': 2, 'question': 'Do you ship internationally?', 'answer': 'Yes, we ship to over 50 countries worldwide. International shipping typically takes 7-14 business days and costs vary by destination. Please note that customs fees may apply.'}, {'id': 3, 'question': 'What payment methods do you accept?', 'answer': 'We accept Visa, Mastercard, American Express, PayPal, and Apple Pay. All payments are processed securely through our encrypted payment system.'}, {'id': 4, 'question': 'How do I track my order?', 'answer': "Once your order ships, you'll receive a tracking number via email. You can also log into your account to vie